<a href="https://colab.research.google.com/github/alanwuha/ce7455-nlp/blob/master/index.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# How to check if PyTorch is using the GPU?

Reference: [How to check if pytorch is using the gpu](https://stackoverflow.com/questions/48152674/how-to-check-if-pytorch-is-using-the-gpu)

In [0]:
import torch

In [7]:
torch.cuda.current_device()

0

In [3]:
torch.cuda.device(0)

In [4]:
torch.cuda.device_count()

1

In [5]:
torch.cuda.get_device_name(0)

'Tesla T4'

In [6]:
torch.cuda.is_available()

True

This tells us that the GPU `Tesla T4` is being used by `PyTorch`.